In [1]:
import joblib
import os
from API.utils import *
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

In [2]:
os.listdir('./model')

['eva.pkl',
 '.ipynb_checkpoints',
 'transformer(23_to_54).pkl',
 'ANN(energy).pkl',
 'transformer(33_to_15).pkl',
 'transformer(SP162_to_Y162).pkl',
 'transformer(54_to_33).pkl',
 'ANN(58_to_sp162).pkl',
 'transformer(43_to_33).pkl',
 'SVR(4_to_23).pkl']

In [3]:
os.listdir('./data/phase_2/cleaned')

['col_names.pkl', 'train.csv', 'train(energy).csv']

# load pretrained model

In [4]:
A = joblib.load('./model/SVR(4_to_23).pkl')
B = joblib.load('./model/transformer(23_to_54).pkl')
C = joblib.load('./model/ANN(58_to_sp162).pkl')
D = joblib.load('./model/transformer(SP162_to_Y162).pkl')
E = joblib.load('./model/ANN(energy).pkl')
F = joblib.load('./model/transformer(54_to_33).pkl')
G = joblib.load('./model/transformer(33_to_15).pkl')
H = joblib.load('./model/transformer(43_to_33).pkl')
col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')

In [5]:
col_names.keys()

dict_keys(['xna', 'xle', 'xhc', 'xhe', 'fna', 'fle', 'fhc', 'fhe', 'sle', 'shc', 'she', 'case', 'duty', 'Rate_m3', 'Rate_ton'])

# A

In [6]:
A_X = pd.read_csv('./data/phase_1/test_170.csv',index_col=0).sample(1)[['T10','T50','T90','N+A']]
A_X.head()

,T10,T50,T90,N+A
60,105.0,120.9,144.2,30.992


In [7]:
A_pred = A.predict(A_X)
print(A_pred.sum(axis=1))
A_pred

0    100.0
dtype: float64


,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,0.062352,0.068309,0.098898,3.63202,1.14893,2.111126,0.445518,11.00834,7.968936,6.096724,2.92438,10.166674,10.719312,4.905218,5.609488,6.917638,10.993071,4.44312,3.574137,0.864521,5.457896,0.477745,0.305645


# B

In [8]:
B_pred = B(A_pred)
print(B_pred.sum(axis=1))
B_pred.head()

0    100.0
dtype: float64


,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,0.0,0.0,0.0,0.003415,0.006831,0.056697,0.062352,0.001366,0.098898,1.125952,0.022979,3.63202,1.369607,0.763319,0.445518,0.741519,0.838729,0.016731,6.350158,0.564346,11.00834,0.0,3.26906,2.263318,0.107193,2.92438,10.612118,10.166674,4.905218,1.332001,0.755595,1.865,1.656893,10.713818,0.279253,6.917638,0.050657,2.66575,1.77737,0.216075,0.927926,1.371369,1.00811,0.917819,1.361815,3.178263,0.477745,0.724963,0.285808,0.137113,0.016616,0.002444,0.00322,0.0


# C

In [9]:
case = pd.read_csv('./data/phase_2/cleaned/train.csv',index_col=0)[col_names['case']].sample(1).reset_index(drop=True)
C_X = case.join(B_pred)
C_X

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_C6P- in Heart Cut (Input)_wt%,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,10.0,100.0,5.0,1.5,0.0,0.0,0.0,0.003415,0.006831,0.056697,0.062352,0.001366,0.098898,1.125952,0.022979,3.63202,1.369607,0.763319,0.445518,0.741519,0.838729,0.016731,6.350158,0.564346,11.00834,0.0,3.26906,2.263318,0.107193,2.92438,10.612118,10.166674,4.905218,1.332001,0.755595,1.865,1.656893,10.713818,0.279253,6.917638,0.050657,2.66575,1.77737,0.216075,0.927926,1.371369,1.00811,0.917819,1.361815,3.178263,0.477745,0.724963,0.285808,0.137113,0.016616,0.002444,0.00322,0.0


In [10]:
C_pred = C.predict(C_X)
print(C_pred.sum(axis=1))
C_pred

0    54.0
dtype: float32


,Individual Component to Light End Split Factor_Oxygen_Ratio,Individual Component to Heart Cut Split Factor_Oxygen_Ratio,Individual Component to Heavy End Split Factor_Oxygen_Ratio,Individual Component to Light End Split Factor_Methane_Ratio,Individual Component to Heart Cut Split Factor_Methane_Ratio,Individual Component to Heavy End Split Factor_Methane_Ratio,Individual Component to Light End Split Factor_Ethane_Ratio,Individual Component to Heart Cut Split Factor_Ethane_Ratio,Individual Component to Heavy End Split Factor_Ethane_Ratio,Individual Component to Light End Split Factor_Propane_Ratio,Individual Component to Heart Cut Split Factor_Propane_Ratio,Individual Component to Heavy End Split Factor_Propane_Ratio,Individual Component to Light End Split Factor_n-Butane_Ratio,Individual Component to Heart Cut Split Factor_n-Butane_Ratio,Individual Component to Heavy End Split Factor_n-Butane_Ratio,Individual Component to Light End Split Factor_i-Pentane_Ratio,Individual Component to Heart Cut Split Factor_i-Pentane_Ratio,Individual Component to Heavy End Split Factor_i-Pentane_Ratio,Individual Component to Light End Split Factor_n-Pentane_Ratio,Individual Component to Heart Cut Split Factor_n-Pentane_Ratio,Individual Component to Heavy End Split Factor_n-Pentane_Ratio,Individual Component to Light End Split Factor_tr2-Pentene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Pentene_Ratio,Individual Component to Heavy End Split Factor_tr2-Pentene_Ratio,Individual Component to Light End Split Factor_Cyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Cyclopentane_Ratio,Individual Component to Heavy End Split Factor_Cyclopentane_Ratio,Individual Component to Light End Split Factor_3-Methylpentane_Ratio,Individual Component to Heart Cut Split Factor_3-Methylpentane_Ratio,Individual Component to Heavy End Split Factor_3-Methylpentane_Ratio,Individual Component to Light End Split Factor_tr2-Hexene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Hexene_Ratio,Individual Component to Heavy End Split Factor_tr2-Hexene_Ratio,Individual Component to Light End Split Factor_n-Hexane_Ratio,Individual Component to Heart Cut Split Factor_n-Hexane_Ratio,Individual Component to Heavy End Split Factor_n-Hexane_Ratio,Individual Component to Light End Split Factor_Methylcyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Methylcyclopentane_Ratio,Individual Component to Heavy End Split Factor_Methylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heart Cut Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heavy End Split Factor_2,2-Dimethylpentane_Ratio",Individual Component to Light End Split Factor_Benzene_Ratio,Individual Component to Heart Cut Split Factor_Benzene_Ratio,Individual Component to Heavy End Split Factor_Benzene_Ratio,Individual Component to Light End Split Factor_Cyclohexane_Ratio,Individual Component to Heart Cut Split Factor_Cyclohexane_Ratio,Individual Component to Heavy End Split Factor_Cyclohexane_Ratio,"Individual Component to Light End Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heart Cut Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heavy End Split Factor_2,2,3-Trimethylbutane_Ratio",Individual Component to Light End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heart Cut Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heavy End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Light End Split Factor_2-Methylhexane_Ratio,Individual Component to Heart Cut Split Factor_2-Methylhexane_Ratio,Individual Component to Heavy End Split Factor_2-Methylhexane_Ratio,"Individual Component to Light End Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heart Cut Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heavy End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Compone

# D

In [11]:
D_pred = D(B_pred,C_pred)
print(D_pred.sum(axis=1))
D_pred

0    300.0
dtype: float64


,Light End Product Properties_Oxygen_wt%,Light End Product Properties_Methane_wt%,Light End Product Properties_Ethane_wt%,Light End Product Properties_Propane_wt%,Light End Product Properties_n-Butane_wt%,Light End Product Properties_i-Pentane_wt%,Light End Product Properties_n-Pentane_wt%,Light End Product Properties_tr2-Pentene_wt%,Light End Product Properties_Cyclopentane_wt%,Light End Product Properties_3-Methylpentane_wt%,Light End Product Properties_tr2-Hexene_wt%,Light End Product Properties_n-Hexane_wt%,Light End Product Properties_Methylcyclopentane_wt%,"Light End Product Properties_2,2-Dimethylpentane_wt%",Light End Product Properties_Benzene_wt%,Light End Product Properties_Cyclohexane_wt%,"Light End Product Properties_2,2,3-Trimethylbutane_wt%",Light End Product Properties_4-Methyl-1-hexene_wt%,Light End Product Properties_2-Methylhexane_wt%,"Light End Product Properties_t-1,3-Dimethylcyclopentane_wt%",Light End Product Properties_n-Heptane_wt%,Light End Product Properties_H2O_wt%,Light End Product Properties_Methylcyclohexane_wt%,Light End Product Properties_Ethylcyclopentane_wt%,"Light End Product Properties_2,3-Dimethyl-1-hexene_wt%",Light End Product Properties_Toluene_wt%,Light End Product Properties_2-Methylheptane_wt%,Light End Product Properties_n-Octane_wt%,Light End Product Properties_Ethylcyclohexane_wt%,Light End Product Properties_EthylBenzene_wt%,Light End Product Properties_P-Xylene_wt%,Light End Product Properties_M-Xylene_wt%,Light End Product Properties_O-Xylene_wt%,Light End Product Properties_2-Methyloctane_wt%,Light End Product Properties_1-Nonene_wt%,Light End Product Properties_n-Nonane_wt%,Light End Product Properties_i-Propylbenzene_wt%,"Light End Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Light End Product Properties_n-Propylcyclohexane_wt%,Light End Product Properties_n-Propylbenzene_wt%,"Light End Product Properties_1,3,5-Trimethylbenzene_wt%",Light End Product Properties_1-Methyl-2-ethylbenzene_wt%,"Light End Product Properties_1,2,3-Trimethylbenzene_wt%","Light End Product Properties_2,4-Dimethyloctane_wt%","Light End Product Properties_2,2,3,3-Tetramethylhexane_wt%",Light End Product Properties_3-Methylnonane_wt%,Light End Product Properties_tert-Butylcyclohexane_wt%,Light End Product Properties_n-Decane_wt%,"Light End Product Properties_1,2-Diethylbenzene_wt%",Light End Product Properties_n-Undecane_wt%,Light End Product Properties_n-Pentylbenzene_wt%,Light End Product Properties_n-Dodecane_wt%,Light End Product Properties_n-Hexylbenzene_wt%,Light End Product Properties_n-Tridecane_wt%,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product

# E

In [12]:
df = pd.read_csv('./data/phase_2/cleaned/train(energy).csv',index_col=0).dropna(axis=0).sample(1)
E_X = df[E.x_col]
E_X

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%,Case Conditions_C7+ in Light End (Input)_wt%,Naphtha Properties_Oxygen_wt%,Naphtha Properties_Methane_wt%,Naphtha Properties_Ethane_wt%,Naphtha Properties_Propane_wt%,Naphtha Properties_n-Butane_wt%,Naphtha Properties_i-Pentane_wt%,Naphtha Properties_n-Pentane_wt%,Naphtha Properties_tr2-Pentene_wt%,Naphtha Properties_Cyclopentane_wt%,Naphtha Properties_3-Methylpentane_wt%,Naphtha Properties_tr2-Hexene_wt%,Naphtha Properties_n-Hexane_wt%,Naphtha Properties_Methylcyclopentane_wt%,"Naphtha Properties_2,2-Dimethylpentane_wt%",Naphtha Properties_Benzene_wt%,Naphtha Properties_Cyclohexane_wt%,"Naphtha Properties_2,2,3-Trimethylbutane_wt%",Naphtha Properties_4-Methyl-1-hexene_wt%,Naphtha Properties_2-Methylhexane_wt%,"Naphtha Properties_t-1,3-Dimethylcyclopentane_wt%",Naphtha Properties_n-Heptane_wt%,Naphtha Properties_H2O_wt%,Naphtha Properties_Methylcyclohexane_wt%,Naphtha Properties_Ethylcyclopentane_wt%,"Naphtha Properties_2,3-Dimethyl-1-hexene_wt%",Naphtha Properties_Toluene_wt%,Naphtha Properties_2-Methylheptane_wt%,Naphtha Properties_n-Octane_wt%,Naphtha Properties_Ethylcyclohexane_wt%,Naphtha Properties_EthylBenzene_wt%,Naphtha Properties_P-Xylene_wt%,Naphtha Properties_M-Xylene_wt%,Naphtha Properties_O-Xylene_wt%,Naphtha Properties_2-Methyloctane_wt%,Naphtha Properties_1-Nonene_wt%,Naphtha Properties_n-Nonane_wt%,Naphtha Properties_i-Propylbenzene_wt%,"Naphtha Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Naphtha Properties_n-Propylcyclohexane_wt%,Naphtha Properties_n-Propylbenzene_wt%,"Naphtha Properties_1,3,5-Trimethylbenzene_wt%",Naphtha Properties_1-Methyl-2-ethylbenzene_wt%,"Naphtha Properties_1,2,3-Trimethylbenzene_wt%","Naphtha Properties_2,4-Dimethyloctane_wt%","Naphtha Properties_2,2,3,3-Tetramethylhexane_wt%",Naphtha Properties_3-Methylnonane_wt%,Naphtha Properties_tert-Butylcyclohexane_wt%,Naphtha Properties_n-Decane_wt%,"Naphtha Properties_1,2-Diethylbenzene_wt%",Naphtha Properties_n-Undecane_wt%,Naphtha Properties_n-Pentylbenzene_wt%,Naphtha Properties_n-Dodecane_wt%,Naphtha Properties_n-Hexylbenzene_wt%,Naphtha Properties_n-Tridecane_wt%
221-17,20.0,100.0,2.0,5.0,0.0,0.0,0.0,0.083997,0.166993,1.387944,2.855886,0.032999,0.339986,3.25187,0.065997,3.445863,3.127875,0.607976,0.729971,1.692932,0.668973,0.012999,5.061798,1.472941,6.096756,0.0,8.532659,5.907764,0.064997,3.792848,6.393744,5.407784,8.50866,1.109956,0.629975,1.554938,1.380945,7.653694,0.199992,2.810888,0.030999,3.233871,2.155914,0.130995,0.563978,0.832967,0.611975,0.673973,0.99996,2.333907,0.913963,0.972961,1.219951,0.183993,0.070997,0.003,0.013999,0.0


In [13]:
E_pred = E.predict(E_X)
E_pred

,Duty_E062 3.5K Steam Rate_ton/hr,Duty_Reboiler Heat Duty_Mkcal/hr,NA_density,LE_density,HC_density,HE_density
0,2.489922,4.256742,0.731282,0.679721,0.743607,0.772922


# F

In [14]:
F_X = D_pred[col_names['xhc']]
F_X

,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product Properties_P-Xylene_wt%,Heart Cut Product Properties_M-Xylene_wt%,Heart Cut Product Properties_O-Xylene_wt%,Heart Cut Product Properties_2-Methyloctane_wt%,Heart Cut Product Properties_1-Nonene_wt%,Heart Cut Product Properties_n-Nonane_wt%,Heart Cut Product Properties_i-Propylbenzene_wt%,"Heart Cut Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Heart Cut Product Properties_n-Propylcyclohexane_wt%,Heart Cut Product Properties_n-Propylbenzene_wt%,"Heart Cut Product Properties_1,3,5-Trimethylbenzene_wt%",Heart Cut Product Properties_1-Methyl-2-ethylbenzene_wt%,"Heart Cut Product Properties_1,2,3-Trimethylbenzene_wt%","Heart Cut Product Properties_2,4-Dimethyloctane_wt%","Heart Cut Product Properties_2,2,3,3-Tetramethylhexane_wt%",Heart Cut Product Properties_3-Methylnonane_wt%,Heart Cut Product Properties_tert-Butylcyclohexane_wt%,Heart Cut Product Properties_n-Decane_wt%,"Heart Cut Product Properties_1,2-Diethylbenzene_wt%",Heart Cut Product Properties_n-Undecane_wt%,Heart Cut Product Properties_n-Pentylbenzene_wt%,Heart Cut Product Properties_n-Dodecane_wt%,Heart Cut Product Properties_n-Hexylbenzene_wt%,Heart Cut Product Properties_n-Tridecane_wt%
0,0.0,0.0,0.0,0.000036,0.000132,0.002462,0.003441,0.000067,0.010395,0.272968,0.009662,1.345146,0.753512,0.720818,0.268737,0.690042,0.890319,0.019206,7.372461,0.656856,12.822146,0.0,3.809209,2.637806,0.124922,3.40889,12.355941,11.839126,5.576134,1.504896,0.819622,2.012468,1.630936,10.016539,0.246453,5.825765,0.04299,2.54765,1.44469,0.175982,0.729795,1.117312,0.768819,0.746297,1.090203,2.447676,0.368016,0.547683,0.212446,0.097562,0.011814,0.001707,0.002247,0.0


In [15]:
F_pred = F(F_X)
print(F_pred.sum(axis=1))
F_pred

0    100.0
dtype: float64


,Hydrogen,Methane,Ethane,Propane,n-Butane,i-Butane,C4 Naphthenes,n-Pentane,C5 i-Paraffins,C5 Naphthenes,n-Hexane,C6 i-Paraffins,C6 Naphthenes,Benzene,n-Heptane,C7 i-Paraffins,C7 Naphthenes,Toluene,n-Octane,C8 i-Paraffins,C8 Naphthenes,Ethylbenzene,P-Xylene,M-Xylene,O-Xylene,n-Nonane,C9 i-Paraffins,C9 Naphthenes,C9 Aromtics,C10+ n-Paraffins,C10+ i-Paraffins,C10+ Naphthenes,C10+ Aromtics
0,0.0,0.0,0.0,0.000036,0.000053,0.000073,0.000007,0.003441,0.002529,0.010395,1.345146,0.28263,1.443554,0.268737,12.822146,9.002804,7.103871,3.40889,11.839126,12.480863,5.576134,1.504896,0.819622,2.012468,1.630936,5.825765,10.262992,3.992339,2.834898,0.646952,4.284176,0.368016,0.226507


# G

In [16]:
G_pred = G(F_pred)
print(G_pred.sum(axis=1))
G_pred

0    100.0
dtype: float64


,HYD,NC1,NC2,NC3,PC4,SAP,SAN,BEZ,TOL,ETB,PXY,MXY,OXY,C9A,C10
0,0.0,0.0,0.0,0.000036,0.000125,68.798576,18.49431,0.268737,3.40889,1.504896,0.819622,2.012468,1.630936,2.834898,0.226507


# H

In [17]:
x = np.random.uniform(size=(1,len(H.x_col)))
x = pd.DataFrame(x,columns=H.x_col)
x

,"N+2A in Naphtha Feed, wt%","Chlorine in Fresh Catalyst, wt%","Carbon in Spent Catalyst, %","C6P- in Naphtha Feed, wt%","3R211 Inlet Tem, oC","3R212 Inlet Temp, oC","3R213 Inlet Temp, oC","3R214 Inlet Temp, oC",Feed Hydrogen,Feed Methane,Feed Ethane,Feed Propane,Feed n-Butane,Feed i-Butane,Feed C4 Naphthenes,Feed n-Pentane,Feed C5 i-Paraffins,Feed C5 Naphthenes,Feed n-Hexane,Feed C6 i-Paraffins,Feed C6 Naphthenes,Feed Benzene,Feed n-Heptane,Feed C7 i-Paraffins,Feed C7 Naphthenes,Feed Toluene,Feed n-Octane,Feed C8 i-Paraffins,Feed C8 Naphthenes,Feed Ethylbenzene,Feed P-Xylene,Feed M-Xylene,Feed O-Xylene,Feed n-Nonane,Feed C9 i-Paraffins,Feed C9 Naphthenes,Feed C9 Aromtics,Feed C10+ n-Paraffins,Feed C10+ i-Paraffins,Feed C10+ Naphthenes,Feed C10+ Aromtics,"Product Separator Pressure, kg/cm2_g",H2/HC Ratio
0,0.578988,0.905769,0.262514,0.617644,0.333452,0.462917,0.01948,0.097332,0.151352,0.568553,0.43773,0.91184,0.876137,0.186869,0.733785,0.058977,0.617433,0.036168,0.990391,0.758564,0.438126,0.205049,0.741124,0.675348,0.35339,0.445403,0.441858,0.118897,0.797838,0.569197,0.626123,0.555757,0.479913,0.597385,0.643687,0.586016,0.5283,0.64933,0.337917,0.46015,0.855926,0.762068,0.180035


In [18]:
H.predict(x)

,Product Hydrogen,Product Methane,Product Ethane,Product Propane,Product n-Butane,Product i-Butane,Product C4 Naphthenes,Product n-Pentane,Product C5 i-Paraffins,Product C5 Naphthenes,Product n-Hexane,Product C6 i-Paraffins,Product C6 Naphthenes,Product Benzene,Product n-Heptane,Product C7 i-Paraffins,Product C7 Naphthenes,Product Toluene,Product n-Octane,Product C8 i-Paraffins,Product C8 Naphthenes,Product Ethylbenzene,Product P-Xylene,Product M-Xylene,Product O-Xylene,Product n-Nonane,Product C9 i-Paraffins,Product C9 Naphthenes,Product C9 Aromtics,Product C10+ n-Paraffins,Product C10+ i-Paraffins,Product C10+ Naphthenes,Product C10+ Aromtics
0,-6.992953e+09,1.390829e+10,-3.155619e+10,-3.623449e+10,1.778146e+09,9.359346e+09,2.160063e+07,-1.140573e+10,-2.016730e+10,1.036162e+09,-3.315416e+09,-1.749883e+10,3.822481e+09,-2.761561e+10,3.979943e+09,6.067012e+09,1.869637e+09,-2.222578e+10,5.032820e+09,1.239725e+10,2.430163e+09,-6.507824e+09,-1.790427e+10,8.492216e+10,8.570635e+09,8.058749e+08,4.462596e+09,-4.598053e+08,3.534804e+10,8.980803e+08,1.983322e+09,1.147679e+09,2.044340e+09


# 把全部模組整合到一塊

In [19]:
class EVA(object):
    def __init__(self):
        self.A = joblib.load('./model/SVR(4_to_23).pkl')
        self.B = joblib.load('./model/transformer(23_to_54).pkl')
        self.C = joblib.load('./model/ANN(58_to_sp162).pkl')
        self.D = joblib.load('./model/transformer(SP162_to_Y162).pkl')
        self.E = joblib.load('./model/ANN(energy).pkl')
        self.F = joblib.load('./model/transformer(54_to_33).pkl')
        self.G = joblib.load('./model/transformer(33_to_15).pkl')
        self.H = joblib.load('./model/transformer(43_to_33).pkl')
        self.col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')
    
    def __call__(self,X):
        x4,case4 = X.iloc[:,:4],X.iloc[:,4:]
        # model forward
        self.y23 = self.A.predict(x4) #percent sum = 100
        self.Xna = self.B(self.y23) #percent sum = 100
        self.sp162 = self.C.predict(case4.join(self.Xna)) #percent sum = 1 * 54component = 54
        self.y162 = self.D(self.Xna,self.sp162) #percent sum = 100*3(le,hc,he) = 300
        self.xhc = self.y162[self.col_names['xhc']] #percent sum = 100
        self.xhc33_p = self.F(self.xhc) # percent 
        
        # predict duty and density(ton/m3)
        E_pred = self.E.predict(case4.join(self.Xna))
        self.duty = E_pred.iloc[:,:2]
        self.density = E_pred.iloc[:,2:]
        
        # get split factor
        self.shc = self.sp162[self.col_names['shc']]
        self.sle = self.sp162[self.col_names['sle']]
        self.she = self.sp162[self.col_names['she']]
        
        # fhc_ton = fhc_m3*density
        self.fhc_m3 = case4['Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr']
        self.fhc_ton = self.fhc_m3.values * self.density.iloc[:,2].values
        
        # calculate fle_ton,fhe_ton
        self.fle_ton = self.fhc_ton * ((self.Xna.values@self.sle.values.ravel())/
                                       (self.Xna.values@self.shc.values.ravel()))
        self.fhe_ton = self.fhc_ton * ((self.Xna.values@self.she.values.ravel())/
                                       (self.Xna.values@self.shc.values.ravel()))
        
        # calculate fna
        self.fna_ton = self.fle_ton + self.fhc_ton + self.fhe_ton
        
        # scale fle,fhc,fhe
        self.fle_ton = self.fle_ton*200000/self.fna_ton
        self.fhc_ton = self.fhc_ton*200000/self.fna_ton
        self.fhe_ton = self.fhe_ton*200000/self.fna_ton
        
        # add columns name
        self.fle_ton = pd.Series(self.fle_ton,name=self.col_names['Rate_ton'][1])
        self.fhe_ton = pd.Series(self.fhe_ton,name=self.col_names['Rate_ton'][3])
        
        # scale duty
        self.duty *= 200/self.fna_ton[0]
        
        # xhc33 = fhc_ton*xhc_33_p
        self.xhc33 = self.fhc_ton[0]*self.xhc33_p
        
        # xhe33 = H(someting + xhc33 + something) 
        NpA = x4.iloc[:,-1][0]
        C6Pm = self.y23[['C6NP','C6IP']].sum(axis=1)[0]
        H_input = [NpA,0.942,3.78,C6Pm,517,517,517,515]+self.xhc33.values.ravel().tolist()+[4.798,1.44]
        H_input = np.array([H_input])
        H_input = pd.DataFrame(H_input,columns=H.x_col)
        self.xhe33 = self.H.predict(H_input)
        
        # final output
        self.y15 = self.G(self.xhe33)
        
        # raname and combine return to user
        self.predict = self.y15.join(self.fle_ton).join(self.fhe_ton).join(self.duty)
        self.naphtha = self.y23
        self.pre_d = self.sp162
        self.reform = self.xhe33

In [20]:
eva = EVA()

# save eva

In [21]:
joblib.dump(eva,'./model/eva.pkl')
del eva

# load eva

In [22]:
eva = joblib.load('./model/eva.pkl')

# create input X for eva

In [23]:
X = pd.DataFrame(np.hstack((A_X.values,case.values)))
X.columns = A_X.columns.tolist()+case.columns.tolist()
X.head()

,T10,T50,T90,N+A,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_C6P- in Heart Cut (Input)_wt%
0,105.0,120.9,144.2,30.992,10.0,100.0,5.0,1.5


# get eva result

In [24]:
eva(X)

In [25]:
eva.predict

,HYD,NC1,NC2,NC3,PC4,SAP,SAN,BEZ,TOL,ETB,PXY,MXY,OXY,C9A,C10,Light End Product Properties_Light End Prod. Rate_ton/hr,Heavy End Product Properties_Heavy End Product Rate_ton/hr,Duty_E062 3.5K Steam Rate_ton/hr,Duty_Reboiler Heat Duty_Mkcal/hr
0,3.762053e+11,9.218237e+11,-8.543945e+11,-1.325642e+12,3.694764e+12,-3.080612e+12,-3.865880e+11,3.757339e+11,-1.665815e+12,8.628155e+11,6.069251e+11,-1.695149e+12,3.090730e+11,1.069748e+12,7.911056e+11,9928.523404,18569.228169,4.267466,2.147504


In [26]:
eva.naphtha

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,0.062352,0.068309,0.098898,3.63202,1.14893,2.111126,0.445518,11.00834,7.968936,6.096724,2.92438,10.166674,10.719312,4.905218,5.609488,6.917638,10.993071,4.44312,3.574137,0.864521,5.457896,0.477745,0.305645


In [27]:
eva.pre_d

,Individual Component to Light End Split Factor_Oxygen_Ratio,Individual Component to Heart Cut Split Factor_Oxygen_Ratio,Individual Component to Heavy End Split Factor_Oxygen_Ratio,Individual Component to Light End Split Factor_Methane_Ratio,Individual Component to Heart Cut Split Factor_Methane_Ratio,Individual Component to Heavy End Split Factor_Methane_Ratio,Individual Component to Light End Split Factor_Ethane_Ratio,Individual Component to Heart Cut Split Factor_Ethane_Ratio,Individual Component to Heavy End Split Factor_Ethane_Ratio,Individual Component to Light End Split Factor_Propane_Ratio,Individual Component to Heart Cut Split Factor_Propane_Ratio,Individual Component to Heavy End Split Factor_Propane_Ratio,Individual Component to Light End Split Factor_n-Butane_Ratio,Individual Component to Heart Cut Split Factor_n-Butane_Ratio,Individual Component to Heavy End Split Factor_n-Butane_Ratio,Individual Component to Light End Split Factor_i-Pentane_Ratio,Individual Component to Heart Cut Split Factor_i-Pentane_Ratio,Individual Component to Heavy End Split Factor_i-Pentane_Ratio,Individual Component to Light End Split Factor_n-Pentane_Ratio,Individual Component to Heart Cut Split Factor_n-Pentane_Ratio,Individual Component to Heavy End Split Factor_n-Pentane_Ratio,Individual Component to Light End Split Factor_tr2-Pentene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Pentene_Ratio,Individual Component to Heavy End Split Factor_tr2-Pentene_Ratio,Individual Component to Light End Split Factor_Cyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Cyclopentane_Ratio,Individual Component to Heavy End Split Factor_Cyclopentane_Ratio,Individual Component to Light End Split Factor_3-Methylpentane_Ratio,Individual Component to Heart Cut Split Factor_3-Methylpentane_Ratio,Individual Component to Heavy End Split Factor_3-Methylpentane_Ratio,Individual Component to Light End Split Factor_tr2-Hexene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Hexene_Ratio,Individual Component to Heavy End Split Factor_tr2-Hexene_Ratio,Individual Component to Light End Split Factor_n-Hexane_Ratio,Individual Component to Heart Cut Split Factor_n-Hexane_Ratio,Individual Component to Heavy End Split Factor_n-Hexane_Ratio,Individual Component to Light End Split Factor_Methylcyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Methylcyclopentane_Ratio,Individual Component to Heavy End Split Factor_Methylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heart Cut Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heavy End Split Factor_2,2-Dimethylpentane_Ratio",Individual Component to Light End Split Factor_Benzene_Ratio,Individual Component to Heart Cut Split Factor_Benzene_Ratio,Individual Component to Heavy End Split Factor_Benzene_Ratio,Individual Component to Light End Split Factor_Cyclohexane_Ratio,Individual Component to Heart Cut Split Factor_Cyclohexane_Ratio,Individual Component to Heavy End Split Factor_Cyclohexane_Ratio,"Individual Component to Light End Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heart Cut Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heavy End Split Factor_2,2,3-Trimethylbutane_Ratio",Individual Component to Light End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heart Cut Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heavy End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Light End Split Factor_2-Methylhexane_Ratio,Individual Component to Heart Cut Split Factor_2-Methylhexane_Ratio,Individual Component to Heavy End Split Factor_2-Methylhexane_Ratio,"Individual Component to Light End Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heart Cut Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heavy End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Compone

In [28]:
eva.reform

,Product Hydrogen,Product Methane,Product Ethane,Product Propane,Product n-Butane,Product i-Butane,Product C4 Naphthenes,Product n-Pentane,Product C5 i-Paraffins,Product C5 Naphthenes,Product n-Hexane,Product C6 i-Paraffins,Product C6 Naphthenes,Product Benzene,Product n-Heptane,Product C7 i-Paraffins,Product C7 Naphthenes,Product Toluene,Product n-Octane,Product C8 i-Paraffins,Product C8 Naphthenes,Product Ethylbenzene,Product P-Xylene,Product M-Xylene,Product O-Xylene,Product n-Nonane,Product C9 i-Paraffins,Product C9 Naphthenes,Product C9 Aromtics,Product C10+ n-Paraffins,Product C10+ i-Paraffins,Product C10+ Naphthenes,Product C10+ Aromtics
0,3.762053e+11,9.218237e+11,-8.543945e+11,-1.325642e+12,1.512486e+12,2.182278e+12,1.577577e+09,-4.978206e+11,-7.952512e+11,1.190264e+10,3.148593e+11,6.040197e+11,-1.258711e+11,3.757339e+11,-3.203442e+11,-1.113339e+12,-1.222809e+11,-1.665815e+12,-2.559231e+11,-1.125297e+12,-2.350432e+11,8.628155e+11,6.069251e+11,-1.695149e+12,3.090730e+11,-3.167610e+10,-2.167645e+10,-1.471958e+10,1.069748e+12,4.057112e+10,1.196871e+11,9.942418e+10,7.911056e+11
